# 遺伝的アルゴリズムの最終章

遺伝的アルゴリズムのゼミでは，アルゴリズムの概要から基礎的な内容まで遠回りをしながら一旦プログラムを書いてみて，修正するという形をとってきた。最終章では，今までしてきたことを総まとめる形で終えたい

## モジュールをインポートしよう！

今回の「遺伝的アルゴリズム」では
<font color="red">
* Numpy
    * 行列（多次元配列）などの数値計算に便利なツール
* Pandas
    * 表などを扱うのに便利なツール
</font>
の２つを使用した。モジュールの入れ方は

In [1]:
import numpy as np
import pandas as pd

とすればいい。ただし，https://blueqat.com/blueqatcloud を使う者や，Pandasでエラーが出る者は `!pip install openpyxl`を最後の行に入れるといい！

## 今までのプログラムを段階ごとに関数にしよう！

### エクセルデータを読み取り，データを返す関数

In [2]:
def read_excel():
    #Excelデータの整形
    df = pd.read_excel("iden.xlsx", sheet_name="Sheet1")
    df = df.iloc[4:14,1:33]
    df = df.fillna(0)
    df = df.replace("◎",2)
    
    #休日数のテーブルを作成
    holiday = df.iloc[:,31:32].reset_index(drop=True)
    holiday.columns = ["休日数"]
    
    #基礎テーブルの作成
    kiso = df.iloc[:,0:31].reset_index(drop=True)
    kiso.columns = [i+1 for i in range(len(kiso.columns))]
    
    return kiso, holiday

**引数はなし**の関数であることに注意！！

この関数はタプル型で２つのデータを返す！出会い系アプリタップルではないよ！

なので，1つの変数で受け取ると....

In [3]:
test = read_excel()
test

(   1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
 0   2   0   0   2   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 1   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 2   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 3   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   2   0   0   
 4   0   0   0   0   0   0   0   0   2   0  ...   0   0   0   0   0   0   0   
 5   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 6   0   0   0   0   0   2   0   0   0   0  ...   0   0   0   0   0   0   0   
 7   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
 8   0   0   0   0   0   0   0   0   0   0  ...   2   0   0   0   0   0   0   
 9   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   2   2   0   
 
    29  30  31  
 0   0   0   0  
 1   0   0   0  
 2   0   0   0  
 3   0   0   0  
 4   0   0   0  
 5   0   0   0  
 6   0   0

１つの変数に２つのデータが入ることになります（汗）

そういうのを意図的に使うのもいいけど，普通は２つの変数で受け取ろう！

In [4]:
kiso, holiday = read_excel()

In [5]:
kiso

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,2,0,0,0,0


In [6]:
holiday

,休日数
0,9
1,8
2,9
3,8
4,9
5,9
6,10
7,8
8,9
9,9


### 「親」の個体を作る関数を作ろう

In [7]:
def FirstCreateGenes(kiso, holiday):
    days = len(kiso.columns)
    kiso_copy = kiso.copy()
    for k in range(len(kiso_copy)):
        h = []
        while len(h) < holiday.iloc[k][0]:
            n = np.random.randint(1, days+1)
            if not n in h:
                h.append(n)
        for i in h:
            if kiso_copy.loc[k,i] == 0:
                kiso_copy.loc[k,i] = 1
    return kiso_copy

#### kiso.columns，kiso.indexって何？

In [8]:
kiso.columns

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
           dtype='int64')

In [9]:
kiso.index

RangeIndex(start=0, stop=10, step=1)

よく分からない文字列が羅列しているけど，表（kisoデータ）の縦列を「kiso.columns」，横列を「kiso.index」で指定しています

さらに，`len`関数でその列の長さを返しています！

では，ここ↓で何をしているか分かりますか？？

```python
h = []
while len(h) < holiday.iloc[k][0]:
    n = np.random.randint(1, days+1)
    if not n in h:
        h.append(n)
```

重複なしの乱数を生成しているんですよ！覚えてますか？

<font color="red">では，この関数使って実際に`kiso_copy`という変数に親の個体を格納しましょう！</font>

In [10]:
kiso_copy = FirstCreateGenes(kiso, holiday)
kiso_copy

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,1,0,2,0,0,0,0,0,1,...,0,0,0,1,0,0,1,1,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,1,1,0
2,0,0,1,0,0,1,1,0,1,1,...,0,1,0,1,0,0,1,0,0,0
3,0,0,0,1,0,1,0,0,1,0,...,0,1,0,0,2,0,1,0,0,0
4,0,1,1,0,0,1,1,1,2,0,...,0,0,1,0,0,0,0,0,0,1
5,1,0,0,0,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
6,1,1,0,0,1,2,1,0,0,0,...,0,0,0,0,0,0,0,1,1,1
7,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
8,0,0,1,0,0,0,0,0,0,0,...,2,0,0,1,0,1,0,0,1,0
9,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,2,2,0,0,1,0


ただ，休日数にずれがありますね（汗）

In [11]:
print(np.count_nonzero(kiso_copy.iloc[0:1])) #親個体に含まれる休日数
print(holiday.loc[0][0]) #必要休日数

11
9


なので，**修正**しましょう！

### 修正関数

In [12]:
def HolidayFix(kiso_copy, holiday):
    days = len(kiso.columns)
    for k in range(len(kiso_copy)):
        if np.count_nonzero(kiso_copy.iloc[k:k+1]) != holiday.loc[k][0]:
            s = np.count_nonzero(kiso_copy.iloc[k:k+1]) - holiday.loc[k][0]
            buf = 0
            if s > 0:
                while buf < s:
                    n = np.random.randint(1,days+1)
                    if kiso_copy.loc[k,n] == 1:
                        buf += 1
                        kiso_copy.loc[k,n] = 0
            #休日数が少ないとき
            else:
                while buf < abs(s):
                    n = np.random.randint(1,days+1)
                    if kiso_copy.loc[k,n] == 0:
                        buf += 1
                        kiso_copy.loc[k,n] = 1
    return kiso_copy

`kiso_copy`をこの`holidayFix`関数に入れて，この関数を通したものを`kiso_copy`の変数にまた格納しましょう！

In [13]:
kiso_copy = HolidayFix(kiso_copy,holiday)
kiso_copy

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,1,0,2,0,0,0,0,0,1,...,0,0,0,1,0,0,1,1,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,0,0,1,0,0,1,1,0,1,1,...,0,1,0,1,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,1,0,...,0,1,0,0,2,0,1,0,0,0
4,0,1,0,0,0,1,1,1,2,0,...,0,0,1,0,0,0,0,0,0,1
5,1,0,0,0,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
6,1,1,0,0,1,2,1,0,0,0,...,0,0,0,0,0,0,0,1,1,1
7,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
8,0,0,1,0,0,0,0,0,0,0,...,2,0,0,1,0,1,0,0,1,0
9,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,2,2,0,0,1,0


In [14]:
print(np.count_nonzero(kiso_copy.iloc[0:1])) #親個体に含まれる休日数
print(holiday.loc[0][0]) #必要休日数

9
9


休日数が同じになりました！Yeah！

次は個体を評価する「**評価関数**」を作ります

In [15]:
def EvaluationFunc(kiso_copy):
    eva = kiso_copy.replace(2,1) # 2を1に置換
    score = 0
    for k in range(len(eva)):
        x = ''.join([str(i) for i in np.array(eva.iloc[k:k+1]).flatten()])
        #5連休以上の評価
        score += sum([((2 - len(i))**2)*-1 for i in x.split('1') if len(i) >= 5 ])

        #3連休以上の評価
        score += sum([((1 - len(i))**2)*-1 for i in x.split('0') if len(i) >= 3])

        #飛び石連休の評価
        score += -10 * (len(x.split('101'))-1)

        #出勤数の評価
    score += sum([abs(len(eva) * 0.7 - (len(eva) - np.sum(eva[k]))) * -4 for k in eva])
    return score

ここ↓は何をしているか覚えてますか？？

```python
 x = ''.join([str(i) for i in np.array(eva.iloc[0:1]).flatten()])
```

思い出してもらいたいのですが，一応。。。

とりあえず，評価にあたって「休日」か「勤務日」かの２択で十分なので，2を1に置換したデータを`eva`変数に格納します

In [16]:
eva = kiso_copy.replace(2,1)
eva

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,1,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,1,1,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,0,0,1,0,0,1,1,0,1,1,...,0,1,0,1,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,1,0,...,0,1,0,0,1,0,1,0,0,0
4,0,1,0,0,0,1,1,1,1,0,...,0,0,1,0,0,0,0,0,0,1
5,1,0,0,0,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
6,1,1,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,1,1
7,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
8,0,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,0
9,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,1,1,0,0,1,0


今回は１行だけ実験します！（<font color="red">こういうのは自分でやろうね</font>）

In [17]:
eva.iloc[0:1]

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,1,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,1,1,0,0


これは縦と横がある２次元データです。

なので，１次元にしたいです。１次元のほうが扱いやすいので。。。。

そこで，`Numpy`にある`flatten`関数で１次元にしたいので，`Numpy`のリストに変換します

In [18]:
np.array(eva.iloc[0:1])

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 0]])

はい，`Numpy`のリストになりました！

まだ，**[[**となってるので２次元であるのが分かります

In [19]:
np.array(eva.iloc[0:1]).flatten()

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0])

はい，１次元にしました

これを`for`文にぶっこみます

In [20]:
for i in np.array(eva.iloc[0:1]).flatten():
    print(i,type(i))

1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>


`type`関数でデータの型も出力したのでリストの一個一個の数字は「整数」を意味する`int`型であることが分かりました！

これらを１つに結合させたいので，`str`関数で数字を「文字」に変換します！

In [21]:
for i in np.array(eva.iloc[0:1]).flatten():
    print(str(i),type(str(i)))

1 <class 'str'>
1 <class 'str'>
0 <class 'str'>
1 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
1 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
1 <class 'str'>
0 <class 'str'>
1 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
0 <class 'str'>
1 <class 'str'>
0 <class 'str'>
0 <class 'str'>
1 <class 'str'>
1 <class 'str'>
0 <class 'str'>
0 <class 'str'>


はい，文字列に成りました！これをリストにするときは。。。

In [22]:
h = []

for i in np.array(eva.iloc[0:1]).flatten():
    h.append(str(i))

h

['1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0']

こんな感じでできます。ただ，なんとこれを一行で書けます！

In [23]:
[str(i) for i in np.array(eva.iloc[0:1]).flatten()]

['1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0']

わーお！

ここで，`join`関数を使うと，リストの文字列を一つの文字列に結合できます！！

In [24]:
'＠'.join([str(i) for i in np.array(eva.iloc[0:1]).flatten()])

'1＠1＠0＠1＠0＠0＠0＠0＠0＠1＠0＠0＠0＠0＠1＠0＠1＠0＠0＠0＠0＠0＠0＠0＠1＠0＠0＠1＠1＠0＠0'

で，クォーテーションの間に入れた文字で結合できます。今回は単に結合したいだけなので何も入れなくていいです

In [25]:
''.join([str(i) for i in np.array(eva.iloc[0:1]).flatten()])

'1101000001000010100000001001100'

で，これを`x`という変数に格納したわけです！

In [26]:
x = ''.join([str(i) for i in np.array(eva.iloc[0:1]).flatten()])

ここで`x`を今度は1で分割してみましょう！

In [27]:
x.split('1')

['', '', '0', '00000', '0000', '0', '0000000', '00', '', '00']

今回では１が休日なので，0の個数が連勤になるわけですね！`len`関数で連勤を調べられるわけです！

それを**評価関数**内でやってるのです！

では，`kiso_copy`のデータを評価してみましょう！

In [28]:
score = EvaluationFunc(kiso_copy)
score

-769.0

わ。。。。ひくいですねww

## ここからは遺伝操作！！

醍醐味の遺伝操作を行います！

遺伝操作をするにあたり親は２体必要です！（当然ですが。。。）１人では遺伝交配はできません！！！！

なので，先程，作った関数を使って`parent`という変数（リスト）に２体入れましょう！

In [29]:
kiso, holiday = read_excel()

parent = []

for i in range(2):
    kiso_copy = FirstCreateGenes(kiso, holiday)
    kiso_copy = HolidayFix(kiso_copy,holiday)
    parent.append(kiso_copy)

In [30]:
parent[0]

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,0,1,2,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
2,0,0,1,1,1,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
3,0,1,0,0,0,1,1,0,0,0,...,1,0,0,0,2,0,0,0,0,1
4,0,0,0,0,0,0,0,1,2,1,...,1,0,1,0,0,0,0,1,1,1
5,0,1,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,1
6,0,0,0,1,0,2,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,1,1,0,0,...,0,1,0,0,1,0,0,0,1,0
8,0,0,1,0,1,0,0,0,0,0,...,2,1,0,1,0,0,1,0,0,0
9,0,0,0,0,0,0,0,1,0,1,...,0,0,1,1,2,2,0,0,0,0


In [31]:
parent[1]

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,0,0,2,0,0,0,0,1,0,...,0,0,1,1,0,0,0,1,1,1
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
2,0,0,1,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,2,0,1,0,0,0
4,0,0,0,0,0,0,0,0,2,0,...,1,0,0,0,0,0,1,1,1,1
5,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,1,1,2,0,0,1,0,...,1,1,0,0,0,0,1,1,0,0
7,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
8,0,0,0,0,0,0,1,0,0,0,...,2,0,0,0,1,1,0,0,0,0
9,0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,2,2,0,0,0,0


ちゃんと入ってますね！

<font color="blue">かんすうべんり〜〜〜〜</font>

### 遺伝交配の関数を定義する

乱数の関数をよく使うので，再定義しよう！

In [32]:
def random():
    return np.random.random()

遺伝交配の関数は。。。

In [33]:
def crossover(ep, sd, p1, p2):
    #ep:50%の一様交差の確率
    #sd:突然変異
    
    days = len(p1.columns)
    
    #一次元化
    p1 = np.array(p1).flatten()
    p2 = np.array(p2).flatten()
    
    #子の変数
    ch1 = []
    ch2 = []
    
    for p1_,p2_ in zip(p1,p2):
        x = True if ep > random() else False
        
        if x == True: #xだけでいい
            ch1.append(p1_)
            ch2.append(p2_)
        else:
            ch1.append(p2_)
            ch2.append(p1_)
    
    ch1, ch2 = mutation(sd, np.array(ch1).flatten(), np.array(ch2).flatten())
    
    ch1 = pd.DataFrame(ch1.reshape(int(len(ch1)/days), days))
    ch2 = pd.DataFrame(ch2.reshape(int(len(ch2)/days), days))
    
    ch1.columns = [i+1 for i in range(len(ch1.columns))]
    ch2.columns = [i+1 for i in range(len(ch1.columns))]
    
    return ch1, ch2

引数４の関数です。

* 引数１つ目は，交差確率（通常は0.5）
* 引数２つ目は，突然変位の確率（好きに決めてね！）
* 引数３つ目は，親１
* 引数４つ目は，親２

だよ！！！

で，乱数は当然，毎回起動するごとに，返す値は違います！

In [34]:
random()

0.20439575703798651

In [35]:
random()

0.678859899938455

In [36]:
random()

0.8028037056608043

なので，その性質を使って変数`x`が50%の確率で，`True`になって50%の確率で`False`になるようにしましょう！

In [37]:
x = True if 0.5 > random() else False
x

False

In [38]:
x = True if 0.5 > random() else False
x

False

In [39]:
x = True if 0.5 > random() else False
x

False

これを使って親の個体の情報を子供１，子供２のどちらに入れるかを決めます！

`True`ならば「親１-> 子１」「親２ -> 子２」

`False`ならば「親１-> 子２」「親２ -> 子１」

に遺伝子を渡すようにしましょう！それが

```python
        if x == True: #xだけでいい
            ch1.append(p1_)
            ch2.append(p2_)
        else:
            ch1.append(p2_)
            ch2.append(p1_)
```

の部分です。

実際は`x == True`じゃなくて`x`で十分です！

分かりやすいようにしてるだけです！

#### zip関数

In [40]:
test_list1 = [i for i in range(10)]
test_list2 = [i for i in range(10)]

In [41]:
test_list1

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [42]:
test_list2

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [43]:
for n,m in zip(test_list1,test_list2):
    print(n,m)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


つまり，２変数を同時に`for`文で回せるようにするのが`zip`関数だと思ってください！

ここ↓の部分は突然変位についての記述のなので後述。
```python
ch1, ch2 = mutation(sd, np.array(ch1).flatten(), np.array(ch2).flatten())
```

今からここ↓について説明します！！

```python
    ch1 = pd.DataFrame(ch1.reshape(int(len(ch1)/days), days))
    ch2 = pd.DataFrame(ch2.reshape(int(len(ch2)/days), days))
    
    ch1.columns = [i+1 for i in range(len(ch1.columns))]
    ch2.columns = [i+1 for i in range(len(ch1.columns))]
```

まず，`reshape`関数は$n$次元から$m$次元に変換する関数です。

#### reshape関数について

In [44]:
a = np.arange(8)
a

array([0, 1, 2, 3, 4, 5, 6, 7])

In [45]:
a.reshape(2,4)

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

他にも便利な機能がたくさんあるけど，調べてね〜！（笑）

まぁつまり，`ch1`は一度，`flatten`関数で一次元になってます。この場合，$10\times 31=310$ のリストになってるはずです。

そこで，310を`days`つまり，31で割ってそれを行数（縦の数）にしてるんですね！で，列数（横の数）はそのまま`days`というわけです！

で，一度，`Pandas`の表データから`Numpy`のリストデータに変換したので，`pd.DataFrame(   )`でまた`Pandas`の表データに戻しているというわけです！

### 突然変位について

In [46]:
def mutation(sd,ch1,ch2):
    x = True if sd > random() else False
    
    if x == True:
        rand = np.random.permutation([i for i in range(len(ch1))])
        rand = rand[:int(len(ch1)//10)]
        for i in rand:
            if ch1[i] == 1:
                ch1[i] == 0
            else:
                ch1[i] == 1

    x = True if sd > random() else False
    
    if x == True:
        rand = np.random.permutation([i for i in range(len(ch2))])
        rand = rand[:int(len(ch2)//10)]
        for i in rand:
            if ch2[i] == 1:
                ch2[i] == 0
            else:
                ch2[i] == 1
    
    return ch1, ch2

#### np.random.permutation()について

ランダムに並び替える関数です

In [47]:
x = np.array([i for i in range(10) ])
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [48]:
np.random.permutation(x)

array([7, 2, 4, 0, 6, 1, 5, 3, 8, 9])

#### リストの4番目以下を取り出す

In [49]:
x[:4]

array([0, 1, 2, 3])

#### リストの４（+1）番目以上を取り出す

In [50]:
x[4:]

array([4, 5, 6, 7, 8, 9])

#### //の演算

In [51]:
5//2

2

In [52]:
10//6

1

In [53]:
20//3

6

つまり，ここ↓では

```python
        rand = np.random.permutation([i for i in range(len(ch1))])
        rand = rand[:int(len(ch1)//10)]
```
では，子１の10%の遺伝子を適当に取りだして，変数`rand`に格納している！

で，ここ↓で遺伝子に変更を加えている！

```python
        for i in rand:
            if ch1[i] == 1:
                ch1[i] == 0
            else:
                ch1[i] == 1
```

### 遺伝交配のカクニン！

In [54]:
kiso, holiday = read_excel()

parent = []

for i in range(2):
    kiso_copy = FirstCreateGenes(kiso, holiday)
    kiso_copy = HolidayFix(kiso_copy,holiday)
    parent.append(kiso_copy)
    
ep = 0.5
sd = 0.05

ch1, ch2 = crossover(ep,sd,parent[0],parent[1])

ch1 = HolidayFix(ch1,holiday)
ch2 = HolidayFix(ch2,holiday)

In [55]:
ch1

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,0,0,2,0,0,1,1,0,1,...,1,1,0,0,0,0,0,0,1,1
1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,1,0,1,0,0,...,0,0,0,1,2,0,0,0,0,0
4,0,0,0,0,0,0,0,0,2,0,...,0,0,1,0,1,0,1,0,1,0
5,1,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,1
6,0,0,1,0,1,2,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
7,1,0,0,0,0,0,1,1,1,0,...,1,0,0,0,0,1,0,1,0,0
8,0,0,0,1,0,0,0,1,0,1,...,2,0,1,0,0,0,0,0,0,1
9,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,2,2,0,0,0,1


In [56]:
ch2

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,2,0,0,2,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,1
1,0,0,1,0,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,1,1
2,1,0,0,0,1,0,0,0,1,0,...,1,0,1,0,1,0,0,1,0,0
3,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,2,0,0,0,0,1
4,0,0,1,0,0,1,0,1,2,0,...,0,0,0,0,1,0,0,0,0,0
5,0,1,0,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
6,0,0,0,1,1,2,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
7,0,1,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,1,0,0,...,2,0,1,0,1,0,0,0,0,0
9,0,0,0,1,0,1,0,0,0,0,...,1,1,0,1,2,2,0,0,0,0


## プログラムを実行してみる！

In [57]:
kiso, holiday = read_excel()

parent = []

for i in range(100):
    kiso_copy = FirstCreateGenes(kiso, holiday)
    kiso_copy = HolidayFix(kiso_copy,holiday)
    score = EvaluationFunc(kiso_copy)
    parent.append([score,kiso_copy])

（効率重視で）親の個体を100個作ってみました！

In [58]:
parent[0]

[-974.0,
    1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
 0   2   0   0   2   0   1   0   0   0   0  ...   0   0   0   0   0   1   0   
 1   0   0   0   0   0   0   0   0   1   1  ...   0   0   0   1   1   1   0   
 2   1   0   0   0   0   0   1   0   1   0  ...   0   0   1   0   0   0   1   
 3   0   0   0   0   0   0   0   0   0   0  ...   0   0   1   1   2   1   0   
 4   0   1   1   0   0   0   0   0   2   0  ...   0   1   0   0   0   0   0   
 5   0   1   0   0   0   0   1   1   0   0  ...   1   1   1   0   0   0   0   
 6   0   0   0   1   0   2   0   0   0   1  ...   1   1   1   1   0   0   1   
 7   0   0   0   0   0   0   0   0   1   0  ...   1   0   0   1   0   0   1   
 8   0   0   1   1   0   1   0   0   0   1  ...   2   0   1   0   1   0   1   
 9   0   0   0   0   0   0   0   1   0   0  ...   1   1   1   1   2   2   0   
 
    29  30  31  
 0   1   0   1  
 1   0   0   0  
 2   0   1   1  
 3   0   0   0  
 4   0   0   0  
 5   0   0   0  
 

In [59]:
parent[0][0]

-974.0

In [60]:
elite_length = 20     #評価関数で点数の良かった20個体のみ残します！
gene_length = 50     #50世代まで遺伝的アルゴリズムを動かします！

ep = 0.5           #交差確率
sd = 0.05          #突然変異の確率

In [61]:
for i, j in parent:
    print(i,j)

-974.0    1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
0   2   0   0   2   0   1   0   0   0   0  ...   0   0   0   0   0   1   0   
1   0   0   0   0   0   0   0   0   1   1  ...   0   0   0   1   1   1   0   
2   1   0   0   0   0   0   1   0   1   0  ...   0   0   1   0   0   0   1   
3   0   0   0   0   0   0   0   0   0   0  ...   0   0   1   1   2   1   0   
4   0   1   1   0   0   0   0   0   2   0  ...   0   1   0   0   0   0   0   
5   0   1   0   0   0   0   1   1   0   0  ...   1   1   1   0   0   0   0   
6   0   0   0   1   0   2   0   0   0   1  ...   1   1   1   1   0   0   1   
7   0   0   0   0   0   0   0   0   1   0  ...   1   0   0   1   0   0   1   
8   0   0   1   1   0   1   0   0   0   1  ...   2   0   1   0   1   0   1   
9   0   0   0   0   0   0   0   1   0   0  ...   1   1   1   1   2   2   0   

   29  30  31  
0   1   0   1  
1   0   0   0  
2   0   1   1  
3   0   0   0  
4   0   0   0  
5   0   0   0  
6   0   0   0  
7   0 

-633.0    1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
0   2   0   1   2   0   1   0   0   0   0  ...   0   1   0   1   0   1   0   
1   1   1   1   0   0   0   0   0   0   0  ...   0   0   1   0   0   1   0   
2   0   1   0   0   1   0   0   0   0   1  ...   1   0   0   0   0   0   1   
3   0   0   0   1   0   0   1   0   0   0  ...   0   0   1   0   2   0   0   
4   0   0   0   0   1   0   0   1   2   0  ...   0   0   0   1   0   0   0   
5   0   1   0   0   0   0   0   1   1   1  ...   0   0   0   1   0   1   1   
6   0   0   0   0   0   2   0   0   1   1  ...   0   1   0   0   1   0   0   
7   1   1   0   0   0   0   0   0   1   0  ...   0   0   0   0   0   1   0   
8   0   1   0   1   0   0   0   0   0   1  ...   2   0   0   0   0   1   0   
9   1   0   0   0   0   1   0   0   1   0  ...   1   0   0   0   2   2   0   

   29  30  31  
0   1   0   0  
1   0   0   0  
2   1   0   0  
3   0   0   0  
4   0   0   1  
5   0   0   0  
6   1   0   0  
7   0 

[10 rows x 31 columns]
-810.0    1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
0   2   0   0   2   0   0   0   0   1   0  ...   1   1   0   0   0   0   0   
1   0   0   1   0   1   1   0   0   0   0  ...   0   0   0   0   0   0   0   
2   0   0   1   1   1   0   0   0   0   1  ...   0   0   0   0   1   0   0   
3   1   0   1   0   0   0   0   0   0   1  ...   0   0   0   0   2   0   0   
4   0   0   0   0   1   0   1   0   2   1  ...   1   0   0   0   0   0   0   
5   0   0   1   1   0   1   0   0   0   0  ...   0   0   0   1   0   0   1   
6   0   0   0   0   0   2   0   0   0   0  ...   1   0   1   0   0   1   1   
7   1   0   1   0   0   0   0   0   0   0  ...   0   0   1   1   0   1   0   
8   1   0   1   1   0   1   0   0   0   0  ...   2   0   0   0   0   0   0   
9   0   0   1   0   0   1   0   0   1   0  ...   0   0   0   1   2   2   0   

   29  30  31  
0   1   0   1  
1   1   0   0  
2   1   1   0  
3   1   0   0  
4   0   1   1  
5   0   0   1  

In [62]:
for i, j in parent:
    print(i)

-974.0
-988.0
-811.0
-742.0
-892.0
-764.0
-792.0
-718.0
-777.0
-722.0
-717.0
-881.0
-834.0
-689.0
-743.0
-852.0
-809.0
-830.0
-687.0
-745.0
-574.0
-767.0
-809.0
-860.0
-814.0
-652.0
-700.0
-985.0
-970.0
-739.0
-1080.0
-797.0
-727.0
-533.0
-631.0
-819.0
-715.0
-790.0
-833.0
-596.0
-780.0
-698.0
-916.0
-682.0
-782.0
-663.0
-576.0
-633.0
-648.0
-916.0
-789.0
-1032.0
-858.0
-899.0
-795.0
-537.0
-627.0
-776.0
-916.0
-964.0
-748.0
-812.0
-738.0
-878.0
-606.0
-617.0
-676.0
-824.0
-710.0
-648.0
-765.0
-728.0
-615.0
-671.0
-763.0
-676.0
-680.0
-753.0
-764.0
-796.0
-805.0
-810.0
-840.0
-732.0
-761.0
-782.0
-797.0
-711.0
-762.0
-605.0
-583.0
-517.0
-860.0
-657.0
-783.0
-930.0
-764.0
-939.0
-698.0
-618.0


#### sorted関数

In [63]:
x = np.random.permutation(np.array([i for i in range(10) ]))
x

array([5, 6, 4, 1, 3, 9, 2, 8, 7, 0])

In [64]:
sorted(x)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [65]:
l = [1, -3, 2]
sorted(l)

[-3, 1, 2]

In [66]:
sorted(l, key=abs)

[1, 2, -3]

In [67]:
l_2d = [[2, 10], [1, -30], [-3, 20]]
l_2d

[[2, 10], [1, -30], [-3, 20]]

In [68]:
sorted(l_2d, key=max)

[[1, -30], [2, 10], [-3, 20]]

引数`key`に最大値を返す組み込み関数`max()`を指定すると，各リストの最大値を基準に並べ替えられる。

In [69]:
sorted(l_2d,key=lambda x:x[0])

[[-3, 20], [1, -30], [2, 10]]

In [70]:
a = lambda x : x*x
a(4)

16

これは`sorted`関数の`key`に

「要素`x`を受け取り，`x[0]`を返す」

という`lambda`式を指定したからです。
これにより，`sorted`関数は，リストの2番目の要素を`key`としてソートを行うようになったのです。

In [71]:
parent = sorted(parent, key=lambda x:-x[0])

In [72]:
for i, j in parent:
    print(i)

-517.0
-533.0
-537.0
-574.0
-576.0
-583.0
-596.0
-605.0
-606.0
-615.0
-617.0
-618.0
-627.0
-631.0
-633.0
-648.0
-648.0
-652.0
-657.0
-663.0
-671.0
-676.0
-676.0
-680.0
-682.0
-687.0
-689.0
-698.0
-698.0
-700.0
-710.0
-711.0
-715.0
-717.0
-718.0
-722.0
-727.0
-728.0
-732.0
-738.0
-739.0
-742.0
-743.0
-745.0
-748.0
-753.0
-761.0
-762.0
-763.0
-764.0
-764.0
-764.0
-765.0
-767.0
-776.0
-777.0
-780.0
-782.0
-782.0
-783.0
-789.0
-790.0
-792.0
-795.0
-796.0
-797.0
-797.0
-805.0
-809.0
-809.0
-810.0
-811.0
-812.0
-814.0
-819.0
-824.0
-830.0
-833.0
-834.0
-840.0
-852.0
-858.0
-860.0
-860.0
-878.0
-881.0
-892.0
-899.0
-916.0
-916.0
-916.0
-930.0
-939.0
-964.0
-970.0
-974.0
-985.0
-988.0
-1032.0
-1080.0


In [73]:
parent = parent[:elite_length]

In [74]:
for i, j in parent:
    print(i)

-517.0
-533.0
-537.0
-574.0
-576.0
-583.0
-596.0
-605.0
-606.0
-615.0
-617.0
-618.0
-627.0
-631.0
-633.0
-648.0
-648.0
-652.0
-657.0
-663.0


#### enumerate関数

In [75]:
test_list = [5,4,8,9,2,4]

In [76]:
test_list

[5, 4, 8, 9, 2, 4]

In [77]:
for n, m in enumerate(test_list):
    print(n,m)

0 5
1 4
2 8
3 9
4 2
5 4


こういうプログラムは

```python
for k1, v1 in enumerate(parent):
        for k2, v2 in enumerate(parent):
            if k1 < k2:
```
**組み合わせ**を考えている！

## さぁ実行しよう！

In [78]:
kiso, holiday = read_excel()

parent = []

for i in range(100):
    kiso_copy = FirstCreateGenes(kiso, holiday)
    kiso_copy = HolidayFix(kiso_copy,holiday)
    score = EvaluationFunc(kiso_copy)
    parent.append([score,kiso_copy])

elite_length = 20
gene_length = 5

ep = 0.5
sd = 0.05

for i in range(gene_length):
    parent = sorted(np.array(parent), key=lambda x:-x[0])
    parent = parent[:elite_length]
    print("第"+str(i+1)+"世代")

    if i == 0:
        top = parent[0]
        
        print(top[0])
        print(np.array(top[1]))
    else:
        if top[0] > parent[0][0]:
            parent.append(top)
            print(top[0])
            print(np.array(top[1]))
            
        else:
            top = parent[0]
            print(parent[0][0])
            print(np.array(parent[0][1]))

    children = []
    
    for k1, v1 in enumerate(parent):
        for k2, v2 in enumerate(parent):
            if k1 < k2:
                ch1, ch2 = crossover(ep,sd,v1[1],v2[1])
                ch1 = HolidayFix(ch1,holiday)
                ch2 = HolidayFix(ch2,holiday)
                
                score1 = EvaluationFunc(ch1)
                score2 = EvaluationFunc(ch2)
                
                children.append([score1,ch1])
                children.append([score2,ch2])

    parent = children.copy()

/Users/daisuke/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


第1世代
-555.0
[[2 0 0 2 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 2 2 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0]
 [1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 2 0 0 0 1 0 0 2 0 0 0 0 0]
 [0 0 1 0 1 0 1 0 2 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0]
 [0 1 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 2 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 2 0 1 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 0 1 2 1 0 0 0 0 0 1 0 0]
 [1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 2 2 0 0 0 0]]
第2世代
-459.0
[[2 1 0 2 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 2 2 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1]
 [0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 2 0 0 0 1 0 0 2 0 0 0 0 0]
 [0 0 1 0 0 1 1 1 2 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0]


## トップを保存しよう！

In [79]:
topshiftdata = top[1].replace(1,"○").replace(2,"◎").replace(0,"")
topshiftdata.to_excel("shift.xlsx")